In [1]:
import tensorflow
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

W0509 10:22:12.345640 21848 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\tf_env\lib\site-packages\tensorflow\python\compat\v2_compat.py:63: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from tensorflow.python.framework import dtypes, random_seed

#DataSet constructor, used to store an monochromatic image based dataset
#dtype can be either uint8 to leave the input as [0, 255], or float32 to rescale into [0, 1].
#Seed arg provides for convenient deterministic testing.
class DataSet(object):
    def __init__(self, images, labels, dtype = dtypes.float32, reshape = True, seed = None):
        seed1, seed2 = random_seed.get_seed(seed)

        #If op level seed is not set, use whatever graph level seed is returned
        numpy.random.seed(seed1 if seed is None else seed2)
        dtype = dtypes.as_dtype(dtype).base_dtype

        if dtype not in (dtypes.uint8, dtypes.float32):
            raise TypeError('Invalid image dtype %r, expected uint8 or float32' % dtype)

        assert images.shape[0] == labels.shape[0], ('images.shape: %s labels.shape: %s' % (images.shape, labels.shape))

        self.num_examples = images.shape[0]

        #Convert shape from [num examples, rows, columns, depth] to [num examples, rows*columns](assuming depth == 1)
        if reshape:
            assert images.shape[3] == 1
            images = images.reshape(images.shape[0], images.shape[1] * images.shape[2])

        #Convert from [0, 255] -> [0.0, 1.0].
        if dtype == dtypes.float32:
            images = images.astype(numpy.float32)
            images = numpy.multiply(images, 1.0 / 255.0)

        self.images = images
        self.labels = labels
        self.epochs_completed = 0
        self.index_in_epoch = 0

    #Return the next batch_size examples from this data set
    def next_batch(self, batch_size, shuffle=True):
        start = self.index_in_epoch

        #Shuffle for the first epoch
        if self.epochs_completed == 0 and start == 0 and shuffle:
            perm0 = numpy.arange(self.num_examples)
            numpy.random.shuffle(perm0)
            self.images = self.images[perm0]
            self.labels = self.labels[perm0]

        #Get Next epoch
        if start + batch_size > self.num_examples:
            #Finished epoch
            self.epochs_completed += 1

            #Get the rest examples in this epoch
            rest_num_examples = self.num_examples - start
            images_rest_part = self.images[start:self.num_examples]
            labels_rest_part = self.labels[start:self.num_examples]

            #Shuffle the data
            if shuffle:
                perm = numpy.arange(self.num_examples)
                numpy.random.shuffle(perm)
                self.images = self.images[perm]
                self.labels = self.labels[perm]

            #Start next epoch
            start = 0
            self.index_in_epoch = batch_size - rest_num_examples
            end = self.index_in_epoch
            images_new_part = self.images[start:end]
            labels_new_part = self.labels[start:end]
            return numpy.concatenate((images_rest_part, images_new_part), axis=0) , numpy.concatenate((labels_rest_part, labels_new_part), axis=0)
        else:
            self.index_in_epoch += batch_size
            end = self.index_in_epoch
            return self.images[start:end], self.labels[start:end]

In [3]:
import collections
import numpy
from random import shuffle
from tensorflow.python.framework import dtypes

Datasets = collections.namedtuple('Datasets', ['train', 'validation', 'test'])

def read_data_semeion(fname = 'data/semeion.data'):
    file = open(fname, 'r')
    lines = file.readlines()

    width = 16
    height = 16
    size = width * height
    classes = 10

    images = [];
    labels = [];
    fnumber = 0;

    for line in lines:
        data = line.split(' ')
        image = [];
        label = [];

        for i in range(0, size):
            image.append(int(float(data[i])))
        images.append(image)

        for i in range(size, size + classes):
            label.append(int(float(data[i]))) 
        labels.append(label)

        fnumber += 1
        
    #Shuffle data
    images_shuffle = []
    labels_shuffle = []
    indexes = list(range(len(images)))
    shuffle(indexes)
    for i in indexes:
        images_shuffle.append(images[i])
        labels_shuffle.append(labels[i])

    images = images_shuffle
    labels = labels_shuffle

    samples = len(lines)
    train_samples = int(0.8*samples)
    test_samples = int(0.2*samples)

    #Train set
    image_train = numpy.array(images[:train_samples], dtype=numpy.uint8)
    image_train = image_train.reshape(train_samples, width, height, 1)

    label_train = numpy.array(labels[:train_samples], dtype=numpy.uint8)

    train = DataSet(image_train, label_train, reshape=True)

    #test set
    image_test = numpy.array(images[test_samples:], dtype=numpy.uint8)
    image_test = image_test.reshape(samples - test_samples, width, height, 1)

    label_test = numpy.array(labels[test_samples:], dtype=numpy.uint8)

    test = DataSet(image_test, label_test, reshape=True)

    return Datasets(train=train, validation=data, test=test)

In [5]:
a_0 = tf.placeholder(tf.float32, [None, 256])
y = tf.placeholder(tf.float32, [None, 10])

middle = 30
w_1 = tf.Variable(tf.truncated_normal([256, middle]))
b_1 = tf.Variable(tf.truncated_normal([1, middle]))
w_2 = tf.Variable(tf.truncated_normal([middle, 10]))
b_2 = tf.Variable(tf.truncated_normal([1, 10]))

In [6]:
def sigma(x):
    return tf.div(tf.constant(1.0),
                  tf.add(tf.constant(1.0), tf.exp(tf.negative(x))))

In [7]:
z_1 = tf.add(tf.matmul(a_0, w_1), b_1)
a_1 = sigma(z_1)
z_2 = tf.add(tf.matmul(a_1, w_2), b_2)
a_2 = sigma(z_2)

W0509 13:45:34.254946 21848 deprecation.py:323] From <ipython-input-6-2b83e92b7715>:3: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [8]:
diff = tf.subtract(a_2, y)

In [9]:
def sigmaprime(x):
    return tf.multiply(sigma(x), tf.subtract(tf.constant(1.0), sigma(x)))

In [10]:
d_z_2 = tf.multiply(diff, sigmaprime(z_2))
d_b_2 = d_z_2
d_w_2 = tf.matmul(tf.transpose(a_1), d_z_2)

d_a_1 = tf.matmul(d_z_2, tf.transpose(w_2))
d_z_1 = tf.multiply(d_a_1, sigmaprime(z_1))
d_b_1 = d_z_1
d_w_1 = tf.matmul(tf.transpose(a_0), d_z_1)

In [11]:
eta = tf.constant(0.5)
step = [
    tf.assign(w_1,
            tf.subtract(w_1, tf.multiply(eta, d_w_1)))
  , tf.assign(b_1,
            tf.subtract(b_1, tf.multiply(eta,
                               tf.reduce_mean(d_b_1, axis=[0]))))
  , tf.assign(w_2,
            tf.subtract(w_2, tf.multiply(eta, d_w_2)))
  , tf.assign(b_2,
            tf.subtract(b_2, tf.multiply(eta,
                               tf.reduce_mean(d_b_2, axis=[0]))))
]

In [12]:
dataset = read_data_semeion()

acct_mat = tf.equal(tf.argmax(a_2, 1), tf.argmax(y, 1))
acct_res = tf.reduce_sum(tf.cast(acct_mat, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(10000):
    batch_x, batch_y = dataset.train.next_batch(10)
    sess.run(step, feed_dict = {a_0: batch_x,
                                y : batch_y})
    if i % 1000 == 0:
        res = sess.run(acct_res, feed_dict =
                       {a_0: dataset.test.images, y: dataset.test.labels})
        print(res)
        tf.train.write_graph(my_graph, path_to_model_pb,
                     'saved_model.pb', as_text=False)

126.0


NameError: name 'my_graph' is not defined

In [19]:
res

570.0